In [1]:
import urllib3
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import plotly.express as px 
import numpy as np 

In [2]:
http = urllib3.PoolManager()
file_name = 'input.txt'
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
response = http.request("GET", url=url)
if response.status == 200:
    with open(file=file_name, mode='wb') as f:
        f.write(response.data)
        print('file download successfully.')
else:
    print(f'file not downloaded. check url!')

file download successfully.


In [3]:
with open(file=file_name, mode='r', encoding='utf-8') as f:
    text = f.read()

print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers
stoi = { ch : i for i, ch in enumerate(chars) }
itos = { i : ch for i, ch in enumerate(chars) }

def encode(s):
    """takes a string, outputs list of integers"""
    return [stoi[ch] for ch in s]

def decode(l):
    """takes list of intergers, returns a string"""
    return ''.join([itos[i] for i in l])

print(encode('hii, there!'))
print(decode(encode('hii, there!')))

[46, 47, 47, 6, 1, 58, 46, 43, 56, 43, 2]
hii, there!


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text))
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n] 
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context.tolist()}, the target is {target}')

When input is [18], the target is 47
When input is [18, 47], the target is 56
When input is [18, 47, 56], the target is 57
When input is [18, 47, 56, 57], the target is 58
When input is [18, 47, 56, 57, 58], the target is 1
When input is [18, 47, 56, 57, 58, 1], the target is 15
When input is [18, 47, 56, 57, 58, 1, 15], the target is 47
When input is [18, 47, 56, 57, 58, 1, 15, 47], the target is 58


In [10]:
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    xb = torch.stack([data[i:block_size+i] for i in ix])
    yb = torch.stack([data[i+1:1+block_size+i] for i in ix])
    return xb, yb 

xb, yb = get_batch('train')
print(xb)
print(xb.shape)
print('---')

print(yb)
print(yb.shape)
print('---')

for b in range(batch_size): # batch dimension
    print(f'---Batch {b+1}---')
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context}, the target is {target}')
    

tensor([[ 1, 39, 50, 51, 53, 57, 58,  1],
        [ 1, 51, 39, 52,  8,  0, 31, 47],
        [39, 58,  1, 44, 53, 56, 58, 46],
        [56, 57,  1, 39, 52, 42,  0, 32]])
torch.Size([4, 8])
---
tensor([[39, 50, 51, 53, 57, 58,  1, 58],
        [51, 39, 52,  8,  0, 31, 47, 56],
        [58,  1, 44, 53, 56, 58, 46, 61],
        [57,  1, 39, 52, 42,  0, 32, 53]])
torch.Size([4, 8])
---
---Batch 1---
When input is tensor([1]), the target is 39
When input is tensor([ 1, 39]), the target is 50
When input is tensor([ 1, 39, 50]), the target is 51
When input is tensor([ 1, 39, 50, 51]), the target is 53
When input is tensor([ 1, 39, 50, 51, 53]), the target is 57
When input is tensor([ 1, 39, 50, 51, 53, 57]), the target is 58
When input is tensor([ 1, 39, 50, 51, 53, 57, 58]), the target is 1
When input is tensor([ 1, 39, 50, 51, 53, 57, 58,  1]), the target is 58
---Batch 2---
When input is tensor([1]), the target is 51
When input is tensor([ 1, 51]), the target is 39
When input is tensor([ 1,

In [11]:
print(xb) #  our input to the transformer

tensor([[ 1, 39, 50, 51, 53, 57, 58,  1],
        [ 1, 51, 39, 52,  8,  0, 31, 47],
        [39, 58,  1, 44, 53, 56, 58, 46],
        [56, 57,  1, 39, 52, 42,  0, 32]])


In [16]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # become (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
logits, loss = model.forward(xb, yb)
print(logits.shape)
print(loss)
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

torch.Size([32, 65])
tensor(4.6015, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJt-wBpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUiQQY:qFINTOBNLI,&oTigq z.c:Cq,SDXzetn3XVjX-YBcHAUhk&PHdhcOb
nhJ?FJU?pRiOLQeUN!BxjPLiq-GJdUV'hsnla!murI!IM?SPNPq?VgC'R
pD3cLv-bxn-tL!upg
SZ!Uvdg CtxtT?hsiW:XxKIiPlagHIsr'zKSVxza?GlDWObPmRJgrIAcmspmZ&viCKot:u3qYXA:rZgv f:3Q-oiwUzqh'Z!I'zRS3SP rVchSFUIdd q?sPJpUdhMCK$VXXevXJFMl,i
YxA:gWId,EXR,iMC,$?srV$VztRwb?KpgUWFjR$zChOLm;JrDnDph
LBj,KZxJa


In [17]:
block_size = 3
batch_size = 2

xb_new, yb_new = get_batch('train')
print(xb_new)
print('---')
print(yb_new)

tensor([[54, 59, 58],
        [53, 60, 43]])
---
tensor([[59, 58,  1],
        [60, 43, 12]])


In [18]:
# "Formula" -> Maximum value in input < num embeddings. Why?
"""
Scenario A: It works
    - Layer: nn.Embedding(65, 65)
        - Vocabulary Size = 65 (Valid indices are 0 to 64)
    - Input: Shape (3, 2). Let's assume the data inside is [[12, 50], [2, 5]]
    - Result: All numbers (12, 50, 2, 5) are less than 65. The lookup succeeds.

Scenario B: It fails
    - Layer: nn.Embedding(10, 10)
        - Vocabulary Size = 10 (Valid indices are 0 to 9)
    - Input: Same input [[12, 50], [2, 5]]
    - Result: The model tries to look up index 12 and 50, but it only has rows 0-9 in its table. It chrashes.
"""
emb = nn.Embedding(61, 61)

In [19]:
emb = nn.Embedding(xb_new.max()+1, xb_new.max()+1)
logis = emb(xb_new)
logits.shape

torch.Size([32, 65])

In [20]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [35]:
batch_size = 32
for step in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.7681589126586914


In [36]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))




I acan tibs NESaieng whe toray MESI haifrall hat ut shoowhe, wowe kist gr maioog,
Y Yon the inchent ud thikehed pere'denl d T:
do's frifilles II mad cthy!
Fan t As who r wathe s thecorred tof pantedst ppry, e wite ivecoup INEN hed len tt gertay, lll ho m, a t g
A core f bl sthane heaig moy-kenkn RE f s iston My bere thamy EY waik porr'd.
Wis
AR:
INons we je ot'd g--gruenf ms; merainortl ru renafiu mat thricoistmu mangs fory
Sie ce LE ty I:
Fourcove, brd wen An allt tor wsit wishoure.
IN:
Prlyo
